In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from metrics import *

import os
import pandas as pd
import numpy as np
import pickle
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
MODEL_NAME = 'cnn'
TOXICITY_COLUMN = 'target'
TEXT_COLUMN = 'comment_text'
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

In [3]:
# Convert taget and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

### load trained model

In [4]:
# All comments must be truncated or padded to be the same length.
MAX_SEQUENCE_LENGTH = 250
def pad_text(texts, tokenizer):
    return pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)

In [5]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
model = load_model('./models/baseline_cnn.h5')
with open('./models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





2024-05-26 14:35:55.011687: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-05-26 14:35:55.123337: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2024-05-26 14:35:55.133691: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5631d143b5c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-05-26 14:35:55.133740: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-05-26 14:35:55.139167: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2024-05-26 14:35:55.395053: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5631d14ba7f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-26 14:35:55.395104: I tensorflow/compil

2024-05-26 14:35:56.468533: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2024-05-26 14:35:56.469334: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1746] Adding visible gpu devices: 0, 1
2024-05-26 14:35:56.469378: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2024-05-26 14:35:56.476216: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1159] Device interconnect StreamExecutor with strength 1 edge matrix:
2024-05-26 14:35:56.476231: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1165]      0 1 
2024-05-26 14:35:56.476237: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1178] 0:   N Y 
2024-05-26 14:35:56.476240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1178] 1:   Y N 
2024-05-26 14:35:56.480704: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH 




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [6]:
test_df = pd.read_csv('./Data/test_private_expanded.csv')
try:
    test_df.rename(columns={'toxicity': 'target'}, inplace=True)
except:
    pass
test_df = convert_dataframe_to_bool(test_df)

In [7]:
test_df[MODEL_NAME] = model.predict(pad_text(test_df[TEXT_COLUMN], tokenizer))[:, 1]

2024-05-26 14:36:11.498352: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2024-05-26 14:36:12.082658: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


In [ ]:
test_df.to_csv(f'./submissions/{MODEL_NAME}_submission.csv')